In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from dotenv import load_dotenv
import json
import os
from typing import List, Dict
import geopandas as gpd
import plotly_express as px
import networkx as nx
from networkx.exception import NetworkXNoPath

load_dotenv()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## TODO
1. Load Aleph entities
2. Load Mapstand data
3. Add missing geometries to mapstand, drop rows that are not used
4. Merge Mapstand with Aleph entities
5. Clean it up
6. Analyse country information and shapes
7. Analyse company information and shapes

In [8]:
PATH_ALEPH = os.environ.get('PATH_WIND_ALEPH')
PATH_RAW = os.environ.get('PATH_RAW')

In [6]:
def parse_json(entities: List) -> pd.DataFrame:
    '''Parses Aleph JSON data
    '''
    
    entity_list = []
    
    for entity in entities:
        data = entity.get('properties')

        for key, value in data.items():
            if isinstance(value, list):
                data.update({key: ','.join(value)})
        entity_id = {'id': entity.get('id')}
        data.update(entity_id)
        entity_list.append(data)
    
    df = pd.DataFrame(entity_list)
    return df


def load_entities(path: str, entity: str) -> pd.DataFrame:
    '''Load entities from Aleph
    (downloaded through alephclient)'''

    entities = []
    with open(f'{path}{entity}.json', 'r') as file:
        for line in file:
            entities.append(json.loads(line))

    df = parse_json(entities)
    return df

## Import Aleph entities

In [9]:
# Import companies

companies = load_entities(PATH_ALEPH, 'companies')
companies.drop(['notes', 'summary', 'sourceUrl','publisher', 'alias', 'description', 
                'leiCode', 'parent', 'amountEur'], axis=1, inplace=True)


# Import and clean assets

assets = load_entities(PATH_ALEPH, 'assets')
assets.dropna(subset='description', inplace=True)
assets.drop(['title', 'authority', 'contractDate', 'jurisdiction', 'registrationNumber',
             'previousName', 'parent', 'leiCode', 'sourceUrl', 'publisher'], axis=1, inplace=True)

# Import ownerships

ownerships = load_entities(PATH_ALEPH, 'ownerships')

# Import legal entities

legalentities = load_entities(PATH_ALEPH, 'legalentities')
legalentities.drop(['idNumber', 'notes',
                    'publisherUrl', 'nationality', 'jurisdiction', 'previousName',
                    'registrationNumber', 'sourceUrl', 'summary', 'publisher', 'alias',
                    'description', 'leiCode', 'parent', 'amountEur'], axis=1, inplace=True)

# Import contracts
contracts = load_entities(PATH_ALEPH, 'contracts')

# Import contract awards

contractawards = load_entities(PATH_ALEPH, 'contractawards')

# Import people

persons = load_entities(PATH_ALEPH, 'persons')

## Import geometries 

We have one geosjon with edited data on capacity and installation year, but it's not complete (I know...). So let's get the newest installed and planned windfarm dataset from MapStand


In [10]:
# Import geometries with edited data

ms = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final.geojson', geometry='geometry')
ms = ms.to_crs(4326)

# So import newest for missing geometries

msn = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_newest_installed.geojson')
msp = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_newest_planned.geojson')
wdz = gpd.GeoDataFrame.from_file(PATH_RAW + 'mapstand_final_wdz.geojson')

# Filter wind development zones
countries = ['Denmark', 'Germany', 'Netherlands', 'Belgium', 'United Kingdom', 'Norway']
wdz = wdz[(wdz.admin_area_name.isin(countries)) & (wdz.mps_est_shore_status=='OFFSHORE')].copy()

to_drop = ['CELTIC OFFSHORE FLOATING WIND AREA A', 'CELTIC OFFSHORE FLOATING WIND AREA B', 'CELTIC OFFSHORE FLOATING WIND AREA C', 'ENECO LUCHTERDUINEN', 'EO1', 'EO2', 'EO2-WEST', 'EO3', 'Ev1',  'Ev2', 'Ev1',
        'Ev10', 'Ev11', 'Ev13', 'Ev14', 'Ev15', 'Ev16', 'Ev17', 'Ev18', 'Ev19', 'Ev2', 'Ev20', 'Ev21', 'Ev22', 'Ev23', 'Ev25', 'Ev27',
        'Ev29', 'Ev31', 'Ev33', 'Ev38', 'Ev40', 'Ev42', 'Ev43', 'Ev44', 'Ev44', 'Ev5', 'Ev7', 'GEMINI I', 'GEMINI II',
        'HOLLANDSE KUST WEST', 'IJMUIDEN VER', 'N-1', 'N-10', 'N-11', 'N-12', 'N-13', 'N-14', 'N-15', 'N-16', 'N-17 a', 'N-17 b', 'N-18 a', 'N-18 b',
        'N-19', 'N-2', 'N-20', 'N-21', 'N-3',  'N-4', 'N-5', 'N-6', 'N-7', 'N-8', 'N-9', 'N2', 'N4', 'NE1', 'NE3', 'NE4', 'NE8', 'NEDSKALERET HESSELO', 'O-1',  'O-1', 'O-1.3 WIND DEVELOPMENT ZONE', 'O-2.2 WIND DEVELOPMENT ZONE', 'O-2.2 WIND DEVELOPMENT ZONE',
        'O-3 WIND DEVELOPMENT ZONE',  'O-4 WIND DEVELOPMENT ZONE','O-6 WIND DEVELOPMENT ZONE', 'PEMBROKESHIRE DEMONSTRATION ZONE',
        'SEO-1', 'W1']

wdz = wdz[~wdz['name'].isin(to_drop)].copy()

# Merge geometries

ms = pd.concat([ms, msn, wdz])

# Create selection of relevant columns

cols = ['mps_uuid', 'name', 'admin_area_name', 'cost_in_million', 'year', 'capacity_mw', 'description', 'remarks',
        'number_generators',  'installation_year', 'mps_est_elevation_max_m', 'mps_est_elevation_min_m', 'geometry']

ms = ms[cols].copy()

In [11]:
ms = ms.sort_values(by=['mps_uuid', 'year'])
ms = ms.drop_duplicates(subset='mps_uuid', keep='first')

In [12]:
# Merge with assets

df = pd.merge(assets,
             ms,
             left_on = 'description',
             right_on='mps_uuid',
             how='left')

# Clean column names

df = df.rename(columns={'name_x': 'name_aleph', 
                        'name_y': 'name_ms',
                        'description_y': 'description',
                        'notes': 'status'
                        })

df.drop('description_x', axis=1, inplace=True)

In [13]:
# Clean MW

df.amount = df.amount.str.replace(' MW', '')
df.amount = df.amount.astype('float')
df['capacity_mw'] = df['capacity_mw'].fillna(df.amount)

# Clean costs
df[['amountEur', 'cost_in_million']]
df.amountEur = df.amountEur.fillna(df.cost_in_million * 1000000).astype('float')

df = df.drop(['cost_in_million', 'amount'], axis=1)

# Clean installation year

df.year = df.year.fillna(df.installation_year.astype(str).replace('.0', '') + '-01-01')

# Clean wind farms we want to leave out and mislabeled wind farm

df = df[(df.name_aleph.notna()) & (df.name_aleph != 'DIAMOND OFFSHORE WIND HOLDINGS I BV')].copy()

In [14]:
# Add simplified status from Marina

st_dict = {'OPERATIONAL': "EXISTING", 
           'EXCLUSIVE_DEVELOPMENT_RIGHTS': "EARLY_PLANS",
           'UNDER_CONSTRUCTION': "EXISTING", 
           'PROJECT_ANNOUNCED': "EARLY_PLANS", 
           'EARLY_PLANNING': "EARLY_PLANS",
           'APPLICATION_SUBMITTED': "SERIOUS_PLANS", 
           'CONSENT_AUTHORISED': "SERIOUS_PLANS", 
           'EXTENSION_REQUESTED': "SERIOUS_PLANS", 
           'CANCELLED': "", 
           'ONSHORE - OPERATIONAL': "",
           'DECOMMISSIONED': "", 
           'AREA_PROPOSED': "EARLY_PLANS", 
           'PLANNING_PERMISSION_EXPIRED': "",
           'COMPETING_PROPOSALS': "",
           'TENDER_PENDING': "",
           'AREA_UNDER_STATE_INVESTIGATION': ""
           }

df['status_simplified'] = df.status.replace(st_dict)

In [15]:
# Create geodataframe

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)

# Reproject so we can calculate area (WGS84/UTM zone 31N is a fine one)

gdf = gdf.to_crs(32631)
gdf['area_km'] = gdf.geometry.area / 1000000

# Write to file
gdf.to_file(PATH_ALEPH + 'analysis_v1.geojson', driver='GeoJSON')

## Analysis

We would like some basic statistics, for instance:
1. Comparisons between countries regarding output, area, number of turbines, if possible normalised with population size.
2. Temporal trends.
3. A detailed overview of company shares in wind farms. For this we would probably need Neo4J to query the ownership graph, or simplify this graph to a table. 
4. Perform linear regressions on costs of wind farms, because some data is lacking. 
5. Assess how much power is sold through PPAs

In [16]:
# First get the population of the countries for normalization, in thousands

population = {'de': 83000,
              'gb': 68000,
              'nl': 18000,
              'be': 12000,
              'no': 5000,
              'dk': 6000}

### Capacity

In [17]:
# Group by country and status simplified

capacity = gdf.groupby(['country', 'status_simplified']).capacity_mw.sum().reset_index()
capacity = capacity[capacity.status_simplified != ''].copy()

# Add capacity per 1000 inhabitants

capacity['cap_pop'] = capacity.capacity_mw.div(capacity.country.map(population))


In [18]:
# Plot

fig = px.bar(capacity, 
             x='country', 
             y='cap_pop', 
             facet_col='status_simplified',
             title = 'Energy output per capita (x1000)')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.show()

### Capacity over time

In [19]:
# Groupby year and country

c_time = gdf.groupby(['country', 'year']).capacity_mw.sum().reset_index() 

In [20]:
fig = px.area(c_time,
              x='year',
              y='capacity_mw',
              color='country',
              title='Projected megawatt production per country per year',
             )
fig.show()

## Analysis ownership structure

There are several ways to go about this. It would be great if we could (partly) automate the generation of ownership tables, using graphs. We could use neo4j for that, or networkx. A query should look something like this:
1. For each asset assign a value of 1
2. Travel on an ownership relation and multiply the weight of that relationship, e.g. 1
3. At the next node find all ownership relationships
4. Travel all relationships and multiply by the weight of that relationship, e.g. .5


One of the problems with ownership is that we have some ranges and non-precise values (e.g. 75+). A solution is to define lower and upper values and convert them to weights. So this means we would create a few extra columns (percentage_lower_bound, percentage_upper_bound) and convert them to proper percentages so we can easily use them for multiplication. 

For now we have to assume that a missing percentage is 100 percent. That will often be te case, but we have to go through the Aleph data again one time to fill in the missing percentages.

One promising approach is to use Dijkstra's Algorithm, the shortest path, between companies (source) and assets (targets). Because we're dealing with a directed graph, this should omit any detours because companies have joint ventures in other projects. 

In [21]:
# Clean percentages

ownerships.percentage = ownerships.percentage.str.replace('+', '-100')
ownerships.percentage.fillna('100', inplace=True)
ownerships.percentage = ownerships.percentage.astype('str')

# Add columns for lower and upper bound

ownerships['perc_lower'] = ownerships.percentage.apply(lambda x: float(x.split('-')[0]) / 100 if '-' in x else float(x) / 100)
ownerships['perc_higher'] = ownerships.percentage.apply(lambda x: float(x.split('-')[1]) / 100 if '-' in x else float(x) / 100)

In [22]:
# Create directed graph

G = nx.DiGraph()

In [23]:
for i, row in assets.iterrows():
    G.add_node(row.id,
               name=row['name'],
               status=row.notes,
               country=row.country, 
               costs=row.amountEur, 
               mps_uuid=row.description,
               aleph_url=row.alephUrl
               )
    
for i, row in companies.iterrows():
    G.add_node(row.id,
               name=row['name'],
               country=row.jurisdiction,
               registration=row.registrationNumber,
               source_url=row.publisherUrl,
               aleph_url=row.alephUrl
               )
    
for i, row in legalentities.iterrows():
    G.add_node(row.id,
               name=row['name'],
               country=row.country,
               aleph_url=row.alephUrl)
    
for i, row in ownerships.iterrows():
    G.add_edge(row.owner,
               row.asset,
               weight_lower=row.perc_lower,
               weight_upper=row.perc_higher,
               percentage=row.percentage,
               source=row.publisherUrl,
               aleph_url=row.alephUrl,
               id=id,
               description=row.description
               )
    




In [24]:
# Get source, target and use Dijkstra's algorithm to get the nodes and relationships inbetween

def get_ownership(graph: nx.Graph, source: str, target: str, weight_range: str) -> Dict:
    '''Calculates ownership percentage
    Parameters:
    ----------
    graph: the graph with assets and owners (G)
    source: precise name of PSC/UBO
    target: precise name of asset
    weight_range: "lower" or "upper"
    '''
    
    # Get source and target node id
    try:
        s = [n for n, v in nx.get_node_attributes(graph, 'name').items() if v == source][0]
        t = [n for n, v in nx.get_node_attributes(graph, 'name').items() if v == target][0]
    except:
        return print(f'could not find entry for {source} and {target}')

    # Calculate shortest path
    try:
        [x for x in nx.dijkstra_path(G, s, t)]
        shortest_path = [x for x in nx.dijkstra_path(G, s, t)]
    except (KeyError, NetworkXNoPath) as error:
        #print(f'No path found between {source} and {target}')
        ownership = {}
        return ownership

    # Get node names for reference
    nodes = [x for x in shortest_path]
    names = [nx.get_node_attributes(G, 'name')[x] for x in nodes]

    # Traverse path and get edge weights
    window_size = 2
    weight = 1
    weights = []

    if weight_range == 'lower':
        weight_range = 'weight_lower'
    elif weight_range == 'upper':
        weight_range = 'weight_upper'
    else:
        raise ValueError('weight should be "lower" or "upper"')

    for i in range(len(shortest_path) - window_size + 1):
        node1 = shortest_path[i: i + window_size][0]
        node2 = shortest_path[i: i + window_size][1]
        nodes = list(G.edges([node1, node2], data=True))
        for node in nodes:
            if node[0] == node1 and node[1] == node2:
                e = node[2][weight_range] 
        weight *= e
        weights.append(e)

    # Create dict
    ownership = {'source_name': source,
                 'source_id': s,
                 'target_name': target,
                 'target_id': t,
                 'name_chain': " -> ".join(names),
                 'edge_weights': weights,
                 'ownership_percentage': weight}

    return ownership



In [25]:
# Load list of ultimate beneficial owners

ubos = pd.read_csv(PATH_ALEPH + 'psc.csv')
ubos = ubos.company.to_list()

In [106]:
bound = 'upper'

# Create a dataframe

o = []
for ubo in ubos:
    for i, row in assets.iterrows():
        ownership = get_ownership(G, ubo, row['name'], bound)
        o.append(ownership)

o = list(filter(None, o))
o = pd.DataFrame(o)

# Add capacity to it

df = pd.merge(o,
              gdf[['name_aleph', 'capacity_mw', 'year', 'status_simplified', 'status', 'country', 'area_km']],
              left_on='target_name',
              right_on='name_aleph',
              how='left')

# Calculate capacity per company

df['output_company_mw'] = df.ownership_percentage * df.capacity_mw
df['company_area_km'] = df.ownership_percentage * df.area_km

# Rename columns

df = df.rename(columns={'source_name': 'company',
                        'target_name': 'asset'})

# Drop duplicates

df = df.drop_duplicates(subset=['company', 'asset']).copy()

In [107]:
# Groupby company

coms = df.groupby('company').agg({'output_company_mw': 'sum',
                                  'area_km': 'sum'})

coms['perc_output'] = coms.output_company_mw / coms.output_company_mw.sum() * 100
coms['perc_area'] = coms.area_km / coms.area_km.sum() * 100
coms.reset_index(inplace=True)

coms.to_csv(PATH_ALEPH + f'north_sea_company_output_and_area_{bound}_bounds.csv', index=False)

In [108]:
# Groupby company, country and status

country = df.groupby(['country', 'company', 'status']).agg({'output_company_mw': 'sum',
                                                             'company_area_km': 'sum'})

country.reset_index(inplace=True)

country.to_csv(PATH_ALEPH + f'north_sea_total_output_grouped_by_country_and_company_and_status_{bound}_bounds.csv', index=False)

In [109]:
# Merge upper and lower

lower = pd.read_csv(PATH_ALEPH + 'north_sea_total_output_grouped_by_country_and_company_and_status_lower_bounds.csv')
upper = pd.read_csv(PATH_ALEPH + 'north_sea_total_output_grouped_by_country_and_company_and_status_upper_bounds.csv')

# Rename

lower = lower.rename(columns={'output_company_mw': 'output_company_mw_lower', 'company_area_km': 'area_km_lower'})
upper = upper.rename(columns={'output_company_mw': 'output_company_mw_upper', 'company_area_km': 'area_km_upper'})

# Merge

df_com = pd.merge(lower,
                 upper,
                 on=['company', 'country', 'status'],
                 how='left')

In [111]:
df_com.to_csv(PATH_ALEPH + 'north_sea_output_company_status_country_lower_and_upper.csv', index=False)

In [ ]:
# Check for endpoints (nodes without incoming edges)

endpoints = [node for node in G.nodes if G.in_degree(node) == 0]

for point in endpoints:
    node = G.nodes[point]
    if 'country' in node.keys():
        
        print(node['name'], node['country'])
    else:
        continue